---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
#spam_data.head(10)

In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
q1_ratio = (spam_data['target'].sum() / len(spam_data['target'])) * 100

In [4]:
def answer_one():
    
    
    return (spam_data['target'].sum() / len(spam_data['target'])) * 100

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import operator


count_vect = CountVectorizer().fit(X_train)

long_word = sorted([(word, len(word)) for word in count_vect.get_feature_names()], \
                   key=operator.itemgetter(1), reverse=True)[0][0]

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import operator


def answer_two():
    
    count_vect = CountVectorizer().fit(X_train)
    longest_word = sorted([(word, len(word)) for word in count_vect.get_feature_names()], \
                          key=operator.itemgetter(1), reverse=True)[0][0]
    
    
    return longest_word


### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score


# Fit the CountVectorizer with train data and transform in
vect_count = CountVectorizer().fit(X_train)
X_train_vectorized = vect_count.transform(X_train)
X_train_vectorized

# Set the Naive Bayes Multinomial
multinomial_nb = MultinomialNB(alpha=0.1)
multinomial_nb.fit(X_train_vectorized, y_train)

# Make prediction and evaluate using ROC-AUC
prediction = multinomial_nb.predict(vect_count.transform(X_test))
auc_score = roc_auc_score(y_test, prediction)

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score


def answer_three():
    
    # Fit the CountVectorizer with train data and transform in
    vect_count = CountVectorizer().fit(X_train)
    X_train_vectorized = vect_count.transform(X_train)

    # Set the Naive Bayes Multinomial
    multinomial_nb = MultinomialNB(alpha=0.1)
    multinomial_nb.fit(X_train_vectorized, y_train)

    # Make prediction and evaluate using ROC-AUC
    prediction = multinomial_nb.predict(vect_count.transform(X_test))
    auc_score = roc_auc_score(y_test, prediction)
    
    
    return auc_score

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Set the TfidfVectorizer() 
tfidf = TfidfVectorizer()

# Fit transform the training data
X_train_vectorized = tfidf.fit_transform(X_train)

# Extract the names from the features
feature_names = tfidf.get_feature_names()

# Estimate the idf using the idf_ method
idf_est = tfidf.idf_

# Save the values in a DataFrame
df = pd.DataFrame({'feature_names': feature_names, 'idf': idf_est})

# Top 20 smallest values
smallest_20_idf = df.sort_values(by='idf', ascending=True, kind='mergesort').head(20).set_index('feature_names')
largest_20_idf = df.sort_values(by='idf', ascending=False, kind='mergesort').head(20).set_index('feature_names')

# Set the format for the correct answer
smallest_20_series = smallest_20_idf['idf']
largest_20_series = largest_20_idf['idf']

In [43]:
def answer_four():

   # Set the TfidfVectorizer() 
    tfidf = TfidfVectorizer()

    # Fit transform the training data
    X_train_vectorized = tfidf.fit_transform(X_train)

    # Extract the names from the features
    feature_names = tfidf.get_feature_names()

    # Estimate the idf using the idf_ method
    idf_est = tfidf.idf_

    # Save the values in a DataFrame
    df = pd.DataFrame({'feature_names': feature_names, 'idf': idf_est})

    # Top 20 smallest values
    smallest_20_idf = df.sort_values(by='idf', ascending=True, kind='mergesort').head(20).set_index('feature_names')
    largest_20_idf = df.sort_values(by='idf', ascending=False, kind='mergesort').head(20).set_index('feature_names')

    # Set the format for the correct answer
    smallest_20_series = smallest_20_idf['idf']
    largest_20_series = largest_20_idf['idf']
    
    
    return (smallest_20_series, largest_20_series)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score


# Set, fit and transform the tfidf vectorizer with min_df=3
tfidf = TfidfVectorizer(min_df=3)
X_train_vectorized = tfidf.fit_transform(X_train)

# Set the Naive Bayes Multinomial
multinomial_nb = MultinomialNB(alpha=0.1)
multinomial_nb.fit(X_train_vectorized, y_train)

# Make prediction and evaluate using ROC-AUC
prediction = multinomial_nb.predict(tfidf.transform(X_test))
auc_score = roc_auc_score(y_test, prediction)

auc_score

0.94162436548223349

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score


def answer_five():
        
    # Set, fit and transform the tfidf vectorizer with min_df=3
    tfidf = TfidfVectorizer(min_df=3)
    X_train_vectorized = tfidf.fit_transform(X_train)

    # Set the Naive Bayes Multinomial
    multinomial_nb = MultinomialNB(alpha=0.1)
    multinomial_nb.fit(X_train_vectorized, y_train)

    # Make prediction and evaluate using ROC-AUC
    prediction = multinomial_nb.predict(tfidf.transform(X_test))
    auc_score = roc_auc_score(y_test, prediction)
    
    
    return auc_score

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
# Estimate the amount of characters in the text for every document in the dataframe
spam_data['amount_characters'] = spam_data['text'].apply(lambda x: len(x))

# Filter the spam_data for target == 0 (No spam)
no_spam_mean = np.mean(spam_data[spam_data['target'] == 0]['amount_characters'])

# Filter the spam_data for target == 1 (Spam)
spam_mean = np.mean(spam_data[spam_data['target'] == 1]['amount_characters'])

In [14]:
def answer_six():
    
    
    # Estimate the amount of characters in the text for every document in the dataframe
    spam_data['amount_characters'] = spam_data['text'].apply(lambda x: len(x))

    # Filter the spam_data for target == 0 (No spam)
    no_spam_mean = np.mean(spam_data[spam_data['target'] == 0]['amount_characters'])

    # Filter the spam_data for target == 1 (Spam)
    spam_mean = np.mean(spam_data[spam_data['target'] == 1]['amount_characters'])
    
    
    return (no_spam_mean, spam_mean)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score


# Train
# Set, fit and transform the tfidf vectorizer with min_df=5
tfidf = TfidfVectorizer(min_df=5)
X_train_vectorized = tfidf.fit_transform(X_train)

# Estimate the amount of characters in the text for every document in the train data
X_train_text_length = X_train.str.len()

# Add the text length as a matrix feature
X_train_vectorized_text_length = add_feature(X_train_vectorized, X_train_text_length)


# Test
# Transform the test data with tfidf vectorizer
X_test_vectorized = tfidf.transform(X_test)

# Estimate the amount of characters in the text for every document in the train data
X_test_text_length = X_test.str.len()

# Add the text length as a matrix feature
X_test_vectorized_text_length = add_feature(X_test_vectorized, X_test_text_length)


# Set the Support Vector Classifier
svm = SVC(C=10000).fit(X_train_vectorized_text_length, y_train)

# Make prediction and evaluate using ROC-AUC
predictions = svm.predict(X_test_vectorized_text_length)
auc_score = roc_auc_score(y_test, predictions)

In [17]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score


def answer_seven():
    
    # Train
    # Set, fit and transform the tfidf vectorizer with min_df=3
    tfidf = TfidfVectorizer(min_df=5)
    X_train_vectorized = tfidf.fit_transform(X_train)

    # Estimate the amount of characters in the text for every document in the train data
    X_train_text_length = X_train.str.len()

    # Add the text length as a matrix feature
    X_train_vectorized_text_length = add_feature(X_train_vectorized, X_train_text_length)


    # Test
    # Transform the test data with tfidf vectorizer
    X_test_vectorized = tfidf.transform(X_test)

    # Estimate the amount of characters in the text for every document in the train data
    X_test_text_length = X_test.str.len()

    # Add the text length as a matrix feature
    X_test_vectorized_text_length = add_feature(X_test_vectorized, X_test_text_length)
    
    # Support vector classifier
    # Set the svc
    svm = SVC(C=10000).fit(X_train_vectorized_text_length, y_train)

    # Make prediction and evaluate using ROC-AUC
    predictions = svm.predict(X_test_vectorized_text_length)
    auc_score = roc_auc_score(y_test, predictions)
    
    
    return auc_score

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
# Estimate the amount of digits in every document in the spam_df
spam_data['digit_count'] = spam_data['text'].apply(lambda x: len([char for char in x if char.isdigit()]))

# Filter the spam_data for target == 0 (No spam)
no_spam_mean = np.mean(spam_data[spam_data['target'] == 0]['digit_count'])

# Filter the spam_data for target == 1 (Spam)
spam_mean = np.mean(spam_data[spam_data['target'] == 1]['digit_count'])

In [19]:
def answer_eight():
    
    # Estimate the amount of digits in every document in the spam_df
    spam_data['digit_count'] = spam_data['text'].apply(lambda x: len([char for char in x if char.isdigit()]))

    # Filter the spam_data for target == 0 (No spam)
    no_spam_mean = np.mean(spam_data[spam_data['target'] == 0]['digit_count'])

    # Filter the spam_data for target == 1 (Spam)
    spam_mean = np.mean(spam_data[spam_data['target'] == 1]['digit_count'])

    
    return (no_spam_mean, spam_mean)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


# Train
# Set, fit and transform the tfidf vectorizer with min_df=5 and ngram_range=(1,3)
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3))
X_train_vectorized = tfidf.fit_transform(X_train)

# Estimate the amount of characters in the text for every document in the train data
X_train_text_length = X_train.str.len()

# Estimate the amount of digits in the document on the training data
X_train_text_digits = X_train.apply(lambda x: len([char for char in x if char.isdigit()]))

# Add the text length and digit count to the matrix feature
X_train_vectorized_text_length_digit = add_feature(X_train_vectorized, [X_train_text_length, X_train_text_digits])


# Test
# Transform the test data with tfidf vectorizer
X_test_vectorized = tfidf.transform(X_test)

# Estimate the amount of characters in the text for every document in the train data
X_test_text_length = X_test.str.len()

# Estimate the amount of digits in the document on the training data
X_test_text_digits = X_test.apply(lambda x: len([char for char in x if char.isdigit()]))

# Add the text length and digit count to the matrix feature
X_test_vectorized_text_length_digit = add_feature(X_test_vectorized, [X_test_text_length, X_test_text_digits])


# Logistic Regression
log_reg = LogisticRegression(C=100).fit(X_train_vectorized_text_length_digit, y_train)

# Make prediction and evaluate using ROC-AUC
predictions = log_reg.predict(X_test_vectorized_text_length_digit)
auc_score = roc_auc_score(y_test, predictions)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score


def answer_nine():
    
    # Train
    # Set, fit and transform the tfidf vectorizer with min_df=5 and ngram_range=(1,3)
    tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3))
    X_train_vectorized = tfidf.fit_transform(X_train)

    # Estimate the amount of characters in the text for every document in the train data
    X_train_text_length = X_train.str.len()

    # Estimate the amount of digits in the document on the training data
    X_train_text_digits = X_train.apply(lambda x: len([char for char in x if char.isdigit()]))

    # Add the text length and digit count to the matrix feature
    X_train_vectorized_text_length_digit = add_feature(X_train_vectorized, [X_train_text_length, X_train_text_digits])


    # Test
    # Transform the test data with tfidf vectorizer
    X_test_vectorized = tfidf.transform(X_test)

    # Estimate the amount of characters in the text for every document in the train data
    X_test_text_length = X_test.str.len()

    # Estimate the amount of digits in the document on the training data
    X_test_text_digits = X_test.apply(lambda x: len([char for char in x if char.isdigit()]))

    # Add the text length and digit count to the matrix feature
    X_test_vectorized_text_length_digit = add_feature(X_test_vectorized, [X_test_text_length, X_test_text_digits])


    # Logistic Regression
    log_reg = LogisticRegression(C=100).fit(X_train_vectorized_text_length_digit, y_train)

    # Make prediction and evaluate using ROC-AUC
    predictions = log_reg.predict(X_test_vectorized_text_length_digit)
    auc_score = roc_auc_score(y_test, predictions)
    
    
    return auc_score

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
# Count the number of non characters (\W in regex) in every document
spam_data['non_character_count'] = spam_data['text'].str.findall(r"\W").str.len()

# Filter the spam_data for target == 0 (No spam)
no_spam_mean = np.mean(spam_data[spam_data['target'] == 0]['non_character_count'])

# Filter the spam_data for target == 1 (Spam)
spam_mean = np.mean(spam_data[spam_data['target'] == 1]['non_character_count'])

In [23]:
def answer_ten():
    
    spam_data['non_character_count'] = spam_data['text'].str.findall(r"\W").str.len()

    # Filter the spam_data for target == 0 (No spam)
    no_spam_mean = np.mean(spam_data[spam_data['target'] == 0]['non_character_count'])

    # Filter the spam_data for target == 1 (Spam)
    spam_mean = np.mean(spam_data[spam_data['target'] == 1]['non_character_count'])
    
    
    return (no_spam_mean, spam_mean)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import re


# Train
# Set, fit and transform the tfidf vectorizer with min_df=5, ngram_range=(1,3), analyzer='word_wb'
tfidf = TfidfVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb')
X_train_vectorized = tfidf.fit_transform(X_train)

# Estimate the amount of characters in the text for every document in the train data
X_train_text_length = X_train.str.len()

# Estimate the amount of digits in the document on the training data
X_train_text_digits = X_train.apply(lambda x: len([char for char in x if char.isdigit()]))

# Estimate the number of non-word characters
X_train_non_char = X_train.apply(lambda x: len(re.findall(r"\W", x)))

# Add the text length and digit count to the matrix feature
training_feature_list = [X_train_text_length, X_train_text_digits, X_train_non_char]
X_train_vectorized_text_length_digit = add_feature(X_train_vectorized, training_feature_list)


# Test
# Transform the test data with tfidf vectorizer
X_test_vectorized = tfidf.transform(X_test)

# Estimate the amount of characters in the text for every document in the train data
X_test_text_length = X_test.str.len()

# Estimate the amount of digits in the document on the training data
X_test_text_digits = X_test.apply(lambda x: len([char for char in x if char.isdigit()]))

# Estimate the number of non-word characters
X_test_non_char = X_test.apply(lambda x: len(re.findall(r"\W", x)))

# Add the text length and digit count to the matrix feature
testing_feature_list = [X_test_text_length, X_test_text_digits, X_test_non_char]
X_test_vectorized_text_length_digit = add_feature(X_test_vectorized, testing_feature_list)


# Logistic Regression
log_reg = LogisticRegression(C=100).fit(X_train_vectorized_text_length_digit, y_train)

# Make prediction and evaluate using ROC-AUC
predictions = log_reg.predict(X_test_vectorized_text_length_digit)
auc_score = roc_auc_score(y_test, predictions)


feature_names = np.array(tfidf.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
sorted_logreg_coef_index = log_reg.coef_[0].argsort()
top10_smallest = list(feature_names[sorted_logreg_coef_index[:10]])
top10_largest = list(feature_names[sorted_logreg_coef_index[:-11:-1]])

answer = (auc_score, top10_smallest, top10_largest)

In [28]:
def answer_eleven():
    
    # Train
    # Set, fit and transform the tfidf vectorizer with min_df=5, ngram_range=(1,3), analyzer='word_wb'
    tfidf = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=[2,5])
    X_train_vectorized = tfidf.fit_transform(X_train)

    # Estimate the amount of characters in the text for every document in the train data
    X_train_text_length = X_train.str.len()

    # Estimate the amount of digits in the document on the training data
    X_train_text_digits = X_train.apply(lambda x: len([char for char in x if char.isdigit()]))

    # Estimate the number of non-word characters
    X_train_non_char = X_train.apply(lambda x: len(re.findall(r"\W", x)))

    # Add the text length and digit count to the matrix feature
    training_feature_list = [X_train_text_length, X_train_text_digits, X_train_non_char]
    X_train_vectorized_text_length_digit = add_feature(X_train_vectorized, training_feature_list)


    # Test
    # Transform the test data with tfidf vectorizer
    X_test_vectorized = tfidf.transform(X_test)

    # Estimate the amount of characters in the text for every document in the train data
    X_test_text_length = X_test.str.len()

    # Estimate the amount of digits in the document on the training data
    X_test_text_digits = X_test.apply(lambda x: len([char for char in x if char.isdigit()]))

    # Estimate the number of non-word characters
    X_test_non_char = X_test.apply(lambda x: len(re.findall(r"\W", x)))

    # Add the text length and digit count to the matrix feature
    testing_feature_list = [X_test_text_length, X_test_text_digits, X_test_non_char]
    X_test_vectorized_text_length_digit = add_feature(X_test_vectorized, testing_feature_list)


    # Logistic Regression
    log_reg = LogisticRegression(C=100).fit(X_train_vectorized_text_length_digit, y_train)

    # Make prediction and evaluate using ROC-AUC
    predictions = log_reg.predict(X_test_vectorized_text_length_digit)
    auc_score = roc_auc_score(y_test, predictions)

    # Append the 3 features names in the get_features_names
    feature_names = np.array(tfidf.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    # Sort the logic results data by the coef and extract the index
    sorted_logreg_coef_index = log_reg.coef_[0].argsort()
    
    # Filter the sorted data to return the smallest and the largest resuts
    top10_smallest = list(feature_names[sorted_logreg_coef_index[:10]])
    top10_largest = list(feature_names[sorted_logreg_coef_index[:-11:-1]])

    
    return (auc_score, top10_smallest, top10_largest)

In [29]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])